#### Eduardo Badillo Á A01020716
#### Rafael Díaz Medina A01024592
#### Moisés Uriel Torres A01021323
#### Ernesto Ramírez A01019589

In [1]:
# Import pandas
import pandas as pd
from datetime import datetime

In [2]:
# Import the dataset
logfile = './access_log_format.csv'
df = pd.read_csv(logfile, sep=",")

# Question 1

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7229 entries, 0 to 7228
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   src_ip           7229 non-null   object
 1   userid           7119 non-null   object
 2   username         7119 non-null   object
 3   datetime         7229 non-null   object
 4   method           7229 non-null   object
 5   response_code    7229 non-null   int64 
 6   return_obj_size  7229 non-null   int64 
 7   dest_address     7229 non-null   object
 8   user_agent       7229 non-null   object
 9   country          7229 non-null   object
dtypes: int64(2), object(8)
memory usage: 564.9+ KB


In [4]:
df.head()

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US


First, we analyze the dataset and observe that it has 7229 entries

Then, we declare the values that we are looking for in the dataset, for the Ip, the Datetime and the country

In [5]:
searched_ip = '14.139.187.130'
searched_datetime = '2017-01-01 06:56:17'
searched_country = "US"

After declaring the values, we search for the Ips that match with the ip 14.139.187.130

In [6]:
ip = df['src_ip'] == searched_ip
print("Ips that match: ", df[ip].groupby(['src_ip']).size())

Ips that match:  src_ip
14.139.187.130    3
dtype: int64


We see that the Ip appears 3 times in the dataset and, then, we get the probability of getting a transaction from that Ip by dividing the number of ocurrences between the total number of entries

In [7]:
ip_probability = 3/7229
print("Probability of a transaction from the searched ip: ", ip_probability)

Probability of a transaction from the searched ip:  0.00041499515838981876


Then, we search for the datetimes that match with the date '2017-01-01 06:56:17'

In [8]:
datetimes = df['datetime'] == searched_datetime
print("Datetimes that match: ", df[datetimes].groupby(['datetime']).size())

Datetimes that match:  datetime
2017-01-01 06:56:17    3
dtype: int64


We see that the datetime appears 3 times in the dataset and we get the probability of getting a transaction in that date

In [9]:
datetime_probability = 3/7229
print("Probability of a transaction in the searched datetime: ", datetime_probability)

Probability of a transaction in the searched datetime:  0.00041499515838981876


We also search for the "US" country in the dataset

In [10]:
countries = df['country'] == searched_country
print("Countries that match: ", df[countries].groupby(['country']).size())

Countries that match:  country
US    1806
dtype: int64


We see that the US appears 1806 times in the dataset and we get the probability of getting a transaction from that country

In [11]:
country_probability = 1806/7229
print("Probability of a transaction in the searched country: ", country_probability)

Probability of a transaction in the searched country:  0.2498270853506709


Finally we get the probability of the three events happening at the same time. We get the probability of a transaction coming from the IP '14.139.187.130', at the datetime '2017-01-01 06:56:17' and from the US

In [12]:
final_search = (df['src_ip'] == searched_ip) & (df['datetime'] == searched_datetime) & (df['country'] == searched_country)
print("Entries that match: ", df[final_search].groupby(['src_ip']).size())

Entries that match:  Series([], dtype: int64)


We see that 0 entries match this query, so the probability of that event ocurring would be of 0. This is the answer of question 1.

Also, with this we can conclude that if at any point some transaction matches with that search, it would be an attack, because its probability of ocurring is 0.

# Question 2

In [13]:
# We copy the original dataframe into a new one so we can modify it without problems
df2 = df.copy()
df2.dtypes

src_ip             object
userid             object
username           object
datetime           object
method             object
response_code       int64
return_obj_size     int64
dest_address       object
user_agent         object
country            object
dtype: object

In [14]:
# We transform the datetime field from a string to a datetime data type 
df2['datetime'] = [datetime.strptime(dt, '%Y-%m-%d %H:%M:%S') for dt in df2['datetime']]
df2.dtypes

src_ip                     object
userid                     object
username                   object
datetime           datetime64[ns]
method                     object
response_code               int64
return_obj_size             int64
dest_address               object
user_agent                 object
country                    object
dtype: object

In [15]:
# Transactions are considered successful when the response code is 200
successful_transactions = df2['response_code'] == 200

# We group by the datetime with a frequency of 30 minutes to get all the successful transactions in each interval
intervals = df2[successful_transactions].groupby(pd.Grouper(key='datetime',freq='30Min')).size()
print(intervals)

datetime
2017-01-01 02:00:00     5
2017-01-01 02:30:00     0
2017-01-01 03:00:00     2
2017-01-01 03:30:00     0
2017-01-01 04:00:00    11
                       ..
2017-01-04 00:30:00    17
2017-01-04 01:00:00    19
2017-01-04 01:30:00    33
2017-01-04 02:00:00    26
2017-01-04 02:30:00    33
Freq: 30T, Length: 146, dtype: int64


These results are the total number of successful transactions in each interval

In [16]:
# After getting all the successful transactions per interval, we get the mean of this result

mean = intervals.mean()
print(mean)

23.164383561643834


Here we can see the that the average number of successful transactions in 30 minutes is of 23.16 transactions

In [17]:
standard_deviation = intervals.std()
print(standard_deviation)

20.794239055402947


And we also see that the standard deviation is of 20.79 transactions. So, one standard deviation above the mean would be 43.95 (23.16 + 20.79). We will round this to 44.

With this information we can know if a transaction can be considered suspicious or not. Assuming that one standard deviation above the mean can be considered suspicious we can know that if there are 44 or more successful transactions in an interval, all of the transactions in that interval will be considered suspicious.

In [18]:
df2.head()

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US


In [21]:
# We will loop over all of the transactions going interval by interval to know if the transactions are considered suspicious
dt = "2017-01-01 02:00:00"
dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')

# Counter to know the current 30-minute interval
cont = 0

# For each 30-minute interval we will check if there are 44 or more successul transactions.
if intervals[cont] >= 44 :
    # If there are, the transactions in that interval will be labeled as suspicious
    is_suspicious = True
else:
    is_suspicious = False

susps = []
# Loop over all the rows in the dataset
for index, row in df2.iterrows():
    
    # If the current transaction exceeds the 30-minute interval
    if row['datetime'] >= (dt + pd.Timedelta(pd.offsets.Minute(30))):
        # Update the counter
        cont += 1
        # Update the current interval
        dt += pd.Timedelta(pd.offsets.Minute(30))
        # Update the suspicious label for the transactions of the current interval
        if intervals[cont] >= 44 :
            is_suspicious = True
        else:
            is_suspicious = False
    
    # For each row we add the the new "is_suspicious" field, that labels the transaction with either True or False
    #df2.set_value(index,'is_suspicious',is_suspicious)
    susps.append(is_suspicious)
    
df2['is_suspicious'] = susps

In [22]:
df2.head()

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country,is_suspicious
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR,False
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR,False
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR,False
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR,False
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US,False


In [23]:
df2.is_suspicious.unique()

array([False,  True])

We can see that the new column was added correctly

In [24]:
df2['is_suspicious'].value_counts()

False    5399
True     1830
Name: is_suspicious, dtype: int64

And we see that in total we have 1830 suspicious transactions

# Question 3

In [25]:
# Copy the dataset
df3 = df2.copy()
df3.head()

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country,is_suspicious
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR,False
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR,False
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR,False
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR,False
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US,False


Using this dataset we will calculate the probability of the previous suspicious classification being legitimate. We will calculate how likely it is to consider a transaction suspicicious based on their user, ip and country. For this, we will use conditional probability:

p( Suspicious | (User ∩ IP ∩ Country) ) = ( p( Suspicious ∩ User ∩ IP ∩ Country) + α) / (p( User ∩ IP ∩ Country) + β)

In [26]:
# We are using alpha and beta so there are no transictions with 0 probability
alpha = 0.99850757
beta = 1

probs = []
# Loop over all the rows in the dataset
for index, row in df3.iterrows():
    
    # Search for the number of transactions that have the same suspicious, ip, user and country that the current row
    first_search = (df3['is_suspicious'] == row['is_suspicious']) & (df3['src_ip'] == row['src_ip']) & (df3['userid'] == row['userid']) & (df3['country'] == row['country'])
    # Calculate the probability by dividing the previous search by the total number of transactions
    first_probability = len(df3.loc[first_search]) / len(df3)
    
    # Search for the number of transactions that have the same ip, user and country that the current row
    second_search = (df3['src_ip'] == row['src_ip']) & (df3['userid'] == row['userid']) & (df3['country'] == row['country'])
    # Calculate the probability by dividing the previous search by the total number of transactions
    second_probability = len(df3.loc[second_search]) / len(df3)
    
    # Get the final probability by substracting 1 minus the result of dividing the first probability by the second one
    probability = 1 - ((first_probability + alpha) / (second_probability + beta))
    
    # Save this probability in a new column of the dataset
    probs.append(probability)

df3['probability'] = probs

In [27]:
df3.head()

,src_ip,userid,username,datetime,method,response_code,return_obj_size,dest_address,user_agent,country,is_suspicious,probability
0,14.139.187.130,hahiss,hahiss@optonline.net,2017-01-01 02:16:51,GET / HTTP/1.1,200,10267,https://www.google.co.in/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR,False,0.001492
1,14.139.187.130,ahuillet,ahuillet@comcast.net,2017-01-01 02:16:55,GET /GitHub-Mark.png HTTP/1.1,200,7428,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,FR,False,0.001492
2,14.139.187.130,gtaylor,gtaylor@gmail.com,2017-01-01 02:16:56,GET /twitter-icon.png HTTP/1.1,200,27788,http://www.secrepo.com/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,BR,False,0.001492
3,68.180.228.229,terjesa,terjesa@sbcglobal.net,2017-01-01 02:17:59,GET /robots.txt HTTP/1.1,200,233,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,FR,False,0.001492
4,68.180.228.229,smallpaul,smallpaul@optonline.net,2017-01-01 02:17:59,GET /self.logs/access.log.2015-11-01.gz HTTP/1.1,200,6465,-,Mozilla/5.0 (compatible; Yahoo! Slurp; http://...,US,False,0.001492


Here we see that the new column was added and all the rows have the probability of the suspicion 